In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests
from bs4 import BeautifulSoup
import time

In [2]:
urls_=["https://www.sofascore.com/real-madrid-chelsea/NsEgb#11137259","https://www.sofascore.com/cadiz-real-madrid/EgbsNOb#10408426","https://www.sofascore.com/real-madrid-chelsea/NsEgb#11137261"]

#### Funcion para extraer todos los datos del performance de un jugador

In [3]:
def obtener_informacion_jugador():
    # Obtiene el contenido HTML de la página
    pagina_html = driver.page_source
    #print(pagina_html)

    # Utiliza BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(pagina_html, 'html.parser')

    print("_______________________________")

    nombre= driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[3]/div/div/div/div[1]/div/div[1]/a/div')
    print(nombre.text)
    puntuacion= driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[3]/div/div/div/div[1]/div/div[2]/div/span')
    print(puntuacion.text)

    print("_______________________________")

    # DEVOLVER TODOS LOS PARÁMETROS DE RENDIMIENTO DEL JUGADOR: encontrar todos los div con la clase "sc-fqkvVR sc-dcJsrY litZes eFJwJL"
    goal_elements = soup.find_all('div', class_='sc-fqkvVR sc-dcJsrY litZes eFJwJL')
    for element in goal_elements:
        # Encuentra el div con la clase "sc-jEACwC hFGVAX" y el span con la clase "sc-jEACwC jnyhQn" dentro de este div
        div_goal = element.find('div', class_='sc-jEACwC hFGVAX')
        span_goal = element.find('span', class_='sc-jEACwC jnyhQn')
        if div_goal and span_goal:
            print(div_goal.text)
            print(span_goal.text)  

#### Obtener jugadores titulares del partido y hacer click sobre todos ellos

###### (para que emerja la tarjeta de cada jugador y posteriormente poder extraer todas las estadísticas de cada jugador en el partido)

In [11]:
# Crea una instancia del controlador del navegador
driver = webdriver.Chrome()

# Navega a la página web que deseas hacer scraping
driver.get("https://www.sofascore.com/real-madrid-chelsea/NsEgb#11137259")

# Espera a que se cargue la página
driver.implicitly_wait(15)
    
# Encuentra el botón de "Consentir" 
button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
# Haz clic en el botón de "Consentir" 
button.click()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Encuentra todos los elementos <a> con la clase 'sc-3937c22d-0 jrbLdB'
divJugadores = driver.find_elements(By.XPATH, '//a[@class="sc-3937c22d-0 jrbLdB"]')

divJugadores[0].click()


"""
#Cerrar ventana
x = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[3]/div[1]/div/div/div[1]/div/div[1]/div/button[2]/svg')
x.click()

# Itera sobre los jugadores y haz click en cada uno
for jugador in divJugadores:
    jugador.click()

#Número jugadres encontrados
divJugadores = soup.find_all('a', class_='sc-3937c22d-0 jrbLdB')
contador = len(divJugadores)
print(f'Número de divs: {contador}')

"""


driver.quit()

<selenium.webdriver.remote.webelement.WebElement (session="b44b0468469f40c9951a08177cc2b5c3", element="8DCCA5E97F26B1785B3BCD29C4FC53DB_element_212")>


In [13]:
# Crea una instancia del controlador del navegador
driver = webdriver.Chrome()

# Navega a la página web que deseas hacer scraping
driver.get("https://www.sofascore.com/real-madrid-chelsea/NsEgb#11137259")

# Espera a que se cargue la página
driver.implicitly_wait(15)

# Encuentra el botón de "Consentir" 
button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
# Haz clic en el botón de "Consentir" 
button.click()

XPATH_array=['//*[@id="__next"]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div/div/div[2]/div[1]/div[1]/a/span[1]']

# Encuentra el botón del Jugador 
button = driver.find_element(By.XPATH,XPATH_array[0] )
# Haz clic en el botón del Jugador 
button.click()

time.sleep(5)

obtener_informacion_jugador()

driver.quit()

_______________________________
Thibaut Courtois
7.5
_______________________________
Minutes played
90'
Saves
3
Goals prevented
1.38
Punches
0
Runs out (succ.)
0 (0)
High claims
0
Saves from inside box
3
Touches
23
Acc. passes
12/13 (92%)
Key passes
0
Crosses (acc.)
0 (0)
Long balls (acc.)
5 (4)
Clearances
0
Blocked shots
0
Interceptions
0
Tackles
0
Dribbled past
0
Ground duels (won)
0 (0)
Aerial duels (won)
0 (0)
Fouls
0
Was fouled
0
Shots on target
0
Shots off target
0
Shots blocked
0
Dribble attempts (succ.)
0 (0)
